In [1]:
from fyp_work.paper_to_equation.Gemini.GeminiPredict import GeminiPredict
from sympy import *
import numpy as np

In [2]:
api_key = "GEMINI_API_KEY"
filename = "Data/HTC_gemini4"
model = "tunedModels/mmltopython4-f3fuppiemnq9"

gp = GeminiPredict(api_key, filename, model)
gp.predict_from_file(filename="Data/HTC_mml") 
gp.save_predictions()

API key loaded successfully!


Generating Predictions:   0%|          | 0/13 [00:00<?, ?row/s]

Generating Predictions: 100%|██████████| 13/13 [00:19<00:00,  1.53s/row]


In [ ]:
class SystemOfEquations():
    def __init__(self, filepath):
        self.str_equations = []
        self.eq = []
        self.filepath = filepath
        self.symbol_lines = set() # Only keep one instance of each symbol line
        self.symbols = set() # Same for symbols

    def extract_equations(self):
        """
        Store the equations in a list of strings
        """
        with open (f"{self.filepath}", "r") as file: 
            for line in file:
                self.str_equations.append(line)

    def get_equations(self):
        """
        Getter for the equation list
        """
        if not self.str_equations:
            self.extract_equations()

        return self.str_equations

    def validation_check(self, equation):
        """
        Check if the equation string is valid SymPy/Python code

        Args:
            equation (str): The equation string to validate

        Returns:
            True if the equation is valid
            False if the equation is invalid
        """
        try:
            eval(equation) # Execute the equation string as code
            return True
        except Exception as e:
            print(e)
            return False

    def parse_equation(self, equation):
        """
        Parse the equation string and store the symbols and equations

        Args:
            equation (str): The equation string to parse
        """
        for line in equation.split("\\n"): # Split equation string into lines (e.g. Symbol, Symbol, Eq)
            if "Symbol" in line:
                line = line.replace('"', "")
                self.symbol_lines.add(line) # Store line for file creation
                self.symbols.add(line.split(" = ")[0]) # Store symbol as a variable
            elif "Eq" in line:
                eq = line.split(" = ")[1]
                eq = eq.replace('"\n', "")
                self.eq.append(sympify(eq)) # Store the equation as a sympy equation
    
    def 







    def solve(self, target):
        pass
    
    def solve_test(self, target):
        symbol_list = {"a", "b", "c"}
        const = {"c"}

        # Define symbols dynamically
        sd = {name: Symbol(name) for name in symbol_list if name not in const}

        # Substitute c = 4 directly in the equations
        c = 4
        # og_eq = [Eq(a, b + 5), Eq(b, c + 1)]
        equations = [Eq(sd["a"], sd["b"] + 5), Eq(sd["b"], c + 1)]
        # equations = [eq.subs({a: sd["a"], b: sd["b"]}) for eq in og_eq]

        # Solve for a
        result = solve(equations)
        print(result[sd[target]])
    
    def solve_test2(self, target):
        pass






def main():
    soe = SystemOfEquations("Data/HTC_gemini5_predictions.txt")
    soe.extract_equations()
    for eq in soe.str_equations:
        soe.parse_equation(eq)
    print(soe.eq)
    


main()

[Eq(h, h_c + h_g), Eq(h, 1.45*k*(p/H)**0.985*tan(θ)/σ), Eq(h, 8000*λ*(K*p/(C*σ_U))**0.86), Eq(h, A*(1 - exp(-B*P))), Eq(h, 2*k_f*k_t*k_w*(1 - A)/(h_f*(-k_f*k_t - k_f*k_w + 2*k_t*k_w))), Eq(h, h_a + h_c + h_l), Eq(h_c, K_st*N_P*α/R), Eq(K_st, 2/(1/k_t + 1/k_s)), Eq(R, sqrt(Rs**2 + Rt**2)), Eq(N_P, 1 - exp(-P*λ/σ_U)), Eq(h_l, K_stl*N_L*β/R), Eq(K_stl, 3/(1/k_t + 1/k_s + 1/k_l)), Eq(N_L, 1 - exp(-γ*δ))]
